In [1]:
import pyproj
import math
import pandas as pd
import geopandas as gpd
import numpy as np
import os

In [2]:
btm = pyproj.CRS('+proj=tmerc +lat_0=0 +lon_0=90 +k=0.9996 +x_0=500000 +y_0=-2000000 +ellps=evrst30 +towgs84=283.729,735.942,261.143,0,0,0,0 +units=m +no_defs')
sph = pyproj.CRS("EPSG:4326")

In [3]:
def WGS_to_BTM(Latitude, Longitude):
    transformer = pyproj.Transformer.from_crs(sph, btm) # Create a transformer object
    X, Y = transformer.transform(Latitude, Longitude) # Perform the transformation
    return X, Y

def BTM_to_WGS(Latitude, Longitude):
    transformer = pyproj.Transformer.from_crs(btm, sph) # Create a transformer object
    X, Y = transformer.transform(Latitude, Longitude) # Perform the transformation
    return X, Y

def get_point_BTM(easting, northing, bearing, distance):
    theta = math.radians(bearing)
    Ep = easting + distance * math.sin(theta)
    Np = northing + distance * math.cos(theta)
    return Ep, Np

In [4]:
gdf = pd.DataFrame()
crs = "EPSG:4326" # WGS
# crs = btm

In [8]:
for folder_path, _, file_names in os.walk('Cross Section'):
    for i, file_name in enumerate(file_names):
        file_path = os.path.join(folder_path, file_name)
        absolute_path = os.path.abspath(file_path)
        df = pd.read_excel(absolute_path)

        latitude = df['LATITUDE'][0]
        longitude = df['LONGITUDE'][0]

        Rl_m = df['RL']
        distances = df['DISTANCE']
        station_id = df['STATION_ID']
        
        easting, northing = WGS_to_BTM(latitude, longitude)
        bearing = -df['Bearing'][0]

        lats_longs = [get_point_BTM(easting, northing, bearing, distance) for distance in distances]
        lats_longs = [BTM_to_WGS(*easting_norting) for easting_norting in lats_longs]
        lats_longs = np.array(lats_longs)
        points = gpd.points_from_xy(lats_longs[:, 1], lats_longs[:, 0], Rl_m, crs=crs)

        temp_df = pd.DataFrame({'geometry': points,
                                'RL': Rl_m,
                                'STATION_ID': station_id,
                                'DISTANCE': distances})
        
        gdf = pd.concat([gdf, temp_df], ignore_index=True)

In [9]:
gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs=crs)
gdf.to_file("points_margere.shp", driver="ESRI Shapefile")

In [ ]:
# import matplotlib.pyplot as plt
# df = pd.read_excel('Cross Section/RMP5.xls')
# rl = df['RL']
# dist = df['DISTANCE']
# plt.plot(dist, rl)

In [6]:
df = pd.read_excel('Demo_Data.xls')

In [19]:
easting = df['Easting']
northing = df['Northing']
rl = df['RL_mPWD']

In [23]:
points = gpd.points_from_xy(northing, easting, rl, crs=sph)

In [24]:
temp_df = pd.DataFrame({'geometry': points,
                                'RL': rl})

In [22]:
gdf = gpd.GeoDataFrame(temp_df, geometry="geometry", crs=sph)
gdf.to_file("points_bathymetry.shp", driver="ESRI Shapefile")